In [1]:
%reload_ext autoreload
%autoreload 2

%gui qt

import numpy as np
import numba as nb
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import napari
from pathlib import Path
import vrExperiment as vre
import vrFunctions as vrf
import basicFunctions as bf
import uiPlottingFunctions as uipf
import redCellGUI as rgui

In [3]:
# Immediate next steps for programming work:

# - red cell processing functions
# - database development
# - ROICaT index alignment tools


In [16]:
mouseName = 'ATL020'
dateString = '2023-05-04'
session = '702'

In [2]:
mouseName = 'CR_Hippocannula6'
dateString = '2022-09-05'
session = '701'

# # vrExperiment registration and preprocessing 
# vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
# vrReg.processTimeline()
# vrReg.processImaging()
# vrReg.processRedCells()
# vrReg.saveParams()
# print(vrReg.sessionPath())
# del vrReg

In [3]:
# vrExperiment registration and preprocessing 
vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
vrReg.doPreprocessing()
vrReg.saveParams()
print(vrReg.sessionPath())
del vrReg

In [3]:
# Load registered vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)

In [13]:
%%time
speedThreshold = 5
omap, smap, lickmap, distvec = vrf.getBehaviorMaps(vrexp,speedThreshold=speedThreshold) # produce smoothed occupancy and speed maps, along with the distance bins used to create them
frameTrialIdx, framePosition, frameSpeed = vrexp.getFrameBehavior() # convert behavioral data into timeframe of spiking data
spkmap = vrf.getSpikeMap(vrexp, frameTrialIdx, framePosition, frameSpeed, distvec, omap, speedThreshold=speedThreshold, standardizeSpks=False, doSmoothing=0) # produce the spkmap 
idxFullTrials = np.all(~np.isnan(omap),axis=1) # find out which trials the mouse explored the whole environment
relmse, relcor = vrf.measureReliability(spkmap[:,idxFullTrials], numcv=3) # measure reliability of spiking
spInfo = vrf.measureSpatialInformation(omap[idxFullTrials], spkmap[:,idxFullTrials]) # measure spatial information of spiking

CPU times: total: 3.58 s
Wall time: 512 ms


In [61]:
# - second, build scrollMatchedImages style GUI with a few extra selections... probably using a Qt Gui application rather than a pyqtgraph layout
# --------- for this, go through each ROI and optionally add manual annotation, which can be saved independently
# --------- it'll be nice to have mechanisms to choose which ROIs to look through, e.g. (1) in order red probability, (2) only red, (3) only ones with manual annotations, (4) etc.... 

# immediate next steps: 
# 1. add manual toggle for Red/NotRed buttons
# 2. add toggle button for showing all manually labeled ROIs
# 3. add toggle button for using just criterion or doing feature criterion first then manual labeling after (in visualization of ROIs)


In [89]:
# -- next steps --
# "switchPlanes" and "saveRedPrms" and anything else...
# if adding in a manual mode for selection of which ROIs are "Red", add an additional colorpanel (or button..?) for classification assignment

# -- further steps -- 
# see if there's a way to click click on the ROI on the viewer screen and flip it to Red / NotRed
# finalize the "saveButton" -- maybe create a new "value" for the vre object to load? 

# Codeblock for developing napari viewer for red cell management
vrexp = vre.vrExperiment(mouseName, dateString, session)
redCell = vre.redCellProcessing(vrexp)
redSelection = rgui.redSelectionGUI(redCell)

In [49]:
# Look into the doubleclick callback, add an annotation to the manualLabels when double clicking
# do some tests to make sure it works as expected!

# Add the buttons for "useManualLabels", "clearManualLabels", and add a button for control-click...
# Add button for visualizing manual labels only, and then add the toggle into the "control-doubleClick" switch for only working if this setting is turned on...


In [86]:
# also need to save the feature cutoff values and optionally load them if they've been created already

In [92]:
np.array(redSelection.featureCutoffs)

array([[ 2.03952226e-07,  9.24995005e-01],
       [-3.71783410e-02,  1.03583564e-01],
       [-2.08535790e-01,  8.69809489e-01],
       [-1.76415304e-01,  3.63010873e-01]])

In [94]:
len(redSelection.featureCutoffs)

4

In [96]:
redSelection.featureNames

['S2P', 'dot()', 'corr()', 'phase-corr']

In [105]:
from copy import copy
for fn in redSelection.featureNames:
    sn = copy(fn)
    print(sn)
    print(sn[0].upper())

S2P
S
dot()
D
corr()
C
phase-corr
P


In [102]:
sn[0].upper()

'S'

In [88]:
redCell.saveone(np.zeros(redCell.value['numROIs']), 'mpciROIs.redCellIdx')
redCell.saveone(np.full((2,redCell.value['numROIs']), False), 'mpciROIs.redCellManualAssignments')